In [98]:
import numpy as np

def f(function, x, x1, x2, d1, d2):
  return eval(function)

def deriv_f(function, x, h, x1, x2, d1, d2):
  return (f(function, x + h, x1, x2, d1, d2) - f(function, x - h, x1, x2, d1, d2))/(2*h)

def deriv2_f(function, x, h, x1, x2, d1, d2):
  return (f(function, x-h, x1, x2, d1, d2) + f(function, x+h, x1, x2, d1, d2) - 2*f(function, x, x1, x2, d1, d2))/(h**2)

def newtonSearch(function, x, x1, x2, d1, d2):
  lmbda = x
  lmbda_next = 0
  e = 10**-6
  h = 10**-6
  timeout = 500
  time = 0

  while time < timeout:
    firstDeriv = deriv_f(function, lmbda, h, x1, x2, d1, d2)
    secondDeriv = deriv2_f(function, lmbda, h, x1, x2, d1, d2)
    lmbda_next = lmbda - (firstDeriv / secondDeriv)

    if abs(lmbda_next - lmbda) < e:
      return [lmbda_next, f(function, lmbda_next, x1, x2, d1, d2)]
    else:
      lmbda = lmbda_next

    time += 1

  raise TimeoutError(f"Algum erro aconteceu, programa demorou muito para responder.")

In [99]:
def hookeJeeves(function, initialValue, dim):
    x1, x2 = initialValue[0], initialValue[1]
    nextValue = np.array(["(x1 + x*d1)", "(x2 + x*d2)"])
    nextFunc = function.replace("x1", nextValue[0]).replace("x2", nextValue[1])

    d = np.array(
        [[1,0], 
        [0,1]]
    )

    limit = 0.01
    k = 0
    timeout = 100
    time = 0
    
    while time < timeout:
        start = np.array([x1, x2])
        end = np.array([0, 0])

        for k in range(dim):          
            d1, d2 = d[k][0], d[k][1]

            [x, y] = newtonSearch(nextFunc, x1, x1, x2, d1, d2)

            x1, x2 = eval(nextValue[0]), eval(nextValue[1])

            if k == dim-1:
                end = np.array([x1, x2])

        resultd = end - start
        startEndMod = np.sqrt(np.sum((resultd) ** 2))

        if startEndMod < limit:
            return end
        
        d1, d2 = resultd[0], resultd[1]

        [x, y] = newtonSearch(nextFunc, x1, x1, x2, d1, d2)

        x1, x2 = eval(nextValue[0]), eval(nextValue[1])

        print(f"Iteração {time} start={start} end={end} ||end-start|| = {startEndMod} d = {resultd} λ^={x}")

        time += 1
    
function = "(x1 - 2)**4 + (x1 - 2*x2)**2"
initialValue = np.array([0,3])

hookeJeeves(function, initialValue, 2)

Iteração 0 start=[0 3] end=[3.1281739  1.56408695] ||end-start|| = 3.44199335094978 d = [ 3.1281739  -1.43591305] λ^=-0.09723359157626962
Iteração 1 start=[2.82401032 1.70370593] end=[2.70540328 1.35270164] ||end-start|| = 0.37050187751020836 d = [-0.11860703 -0.35100429] λ^=0.21863481370358034
Iteração 2 start=[2.67947165 1.27595988] end=[2.41202512 1.20601256] ||end-start|| = 0.2764421739860723 d = [-0.26744653 -0.06994732] λ^=0.5718224369734635
Iteração 3 start=[2.25909319 1.16601511] end=[2.28549186 1.14274593] ||end-start|| = 0.035190116917413646 d = [ 0.02639866 -0.02326918] λ^=-0.21729602539362272
Iteração 4 start=[2.27975553 1.14780223] end=[2.26032178 1.13016089] ||end-start|| = 0.026246665125400356 d = [-0.01943375 -0.01764134] λ^=1.7798119305780313
Iteração 5 start=[2.22573336 1.09876263] end=[2.1848855  1.09244275] ||end-start|| = 0.04133386818726146 d = [-0.04084786 -0.00631988] λ^=0.46777035356156615


array([2.16927259, 1.0846363 ])

In [100]:
# functionx = "(x1+x - 2)**4 + (x1+x - 2*x2)**4"
# functiony = "(x1 - 2)**4 + (x1 - 2*(x2+x))**4"